In [1]:
!git clone -b roshan_b https://github.com/roshant2003/Railway-Track-Fault-Detection-Project
!cp -r Railway-Track-Fault-Detection-Project/Railway/ /content/Dataset
!cp -r Railway-Track-Fault-Detection-Project/Railway_pre/ /content/Dataset_Pre
!pip install tensorflow==2.15

Cloning into 'Railway-Track-Fault-Detection-Project'...
remote: Enumerating objects: 10334, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10334 (delta 1), reused 11 (delta 1), pack-reused 10322 (from 1)
Receiving objects: 100% (10334/10334), 599.19 MiB | 7.79 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (10309/10309), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
     

In [1]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
data_dir = 'Dataset_Pre'

n_classes = 7

# Image parameters
n_channels = 3
image_size = 224

# Model parameters
pooling_avg = 'avg'
dense_layer_activation = 'softmax'
objective_function = 'categorical_crossentropy'

# Training parameters
loss_metrics = ['accuracy']
n_epochs = 100
early_stop_patience = 10

steps_per_epoch_training = 10
steps_per_epoch_validation = 10

batch_size_training = 100
batch_size_validation = 100

batch_size_testing = 1

In [8]:
# define generators
datagen = ImageDataGenerator(
        zoom_range=0.15,
        shear_range=0.15,
        horizontal_flip=True,
        validation_split=0.2,
        rescale=1.0/255)

train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_training,
        class_mode='categorical',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_validation,
        class_mode='categorical',
        subset='validation')

Found 4125 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [9]:
# Define ResNet Model
model = Sequential()
model.add(ResNet50(input_shape=(224, 224, 3), include_top = False, pooling = pooling_avg, weights = 'imagenet'))
model.add(Dense(n_classes, activation = dense_layer_activation))

# Freeze base layers
model.layers[0].trainable = False

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 7)                 14343     
                                                                 
Total params: 23602055 (90.03 MB)
Trainable params: 14343 (56.03 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [10]:
from tensorflow.keras import optimizers

# Compile model
sgd = optimizers.SGD(learning_rate = 0.01, weight_decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = objective_function, metrics = loss_metrics)

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# Prepare early stopping to avoid overfitting
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = early_stop_patience)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [11]:
fit_history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch_training,
        epochs = n_epochs,
        validation_data=validation_generator,
        validation_steps=steps_per_epoch_validation,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

model.load_weights("../working/best")

model.save("RN50_RTFD.keras")

Epoch 1/100
10/10 [==============================] - 60s 6s/step - loss: 2.0210 - accuracy: 0.4920 - val_loss: 3.5586 - val_accuracy: 0.5480
Epoch 2/100
10/10 [==============================] - 51s 5s/step - loss: 1.7469 - accuracy: 0.5810 - val_loss: 3.4907 - val_accuracy: 0.5490
Epoch 3/100
10/10 [==============================] - 53s 6s/step - loss: 2.2719 - accuracy: 0.5670 - val_loss: 1.3130 - val_accuracy: 0.6130
Epoch 4/100
10/10 [==============================] - 54s 6s/step - loss: 0.8896 - accuracy: 0.7351 - val_loss: 0.7853 - val_accuracy: 0.7340
Epoch 5/100
10/10 [==============================] - 34s 4s/step - loss: 0.7921 - accuracy: 0.7514 - val_loss: 0.8636 - val_accuracy: 0.7190
Epoch 6/100
10/10 [==============================] - 52s 6s/step - loss: 0.6807 - accuracy: 0.7610 - val_loss: 0.6810 - val_accuracy: 0.7280
Epoch 7/100
10/10 [==============================] - 31s 3s/step - loss: 0.6472 - accuracy: 0.7578 - val_loss: 0.7394 - val_accuracy: 0.7370
Epoch 8/100
1

In [12]:
# Load TF Keras model
import tensorflow as tf
model = tf.keras.models.load_model("RN50_RTFD.keras")

In [15]:
# Accuracy test with complete dataset
import os
import cv2
import progressbar
import numpy as np

def classify(image):
    # Pre-processing
    image = cv2.resize(image,(224,224))
    image = np.array(np.expand_dims(image,0), dtype=np.float32)
    image = image / 255.
    output = model.predict(image, verbose=0)
    return np.argmax(output, axis=1)[0]

def test_model(folder, classes, length):
    class_idx = 0
    curr_count = 0
    correct_count = 0
    with progressbar.ProgressBar(max_value=length) as bar:
        for c in classes:
            for f in os.listdir(os.path.join(folder, c)):
                bar.update(curr_count)
                image = cv2.imread(os.path.join(folder, c, f))
                out = classify(image)

                if out == class_idx: correct_count += 1
                curr_count += 1
            class_idx += 1

    print(f"Total images           : {curr_count}")
    print(f"Correct predictions    : {correct_count}")
    print(f"Incorrect predictions  : {curr_count-correct_count}")
    print(f"Accuracy               : {correct_count/curr_count*100} %")

In [16]:
# TF Keras
classes = list(train_generator.class_indices.keys())
test_model('Dataset_Pre', classes, 5153)

100% (5153 of 5153) |####################| Elapsed Time: 0:07:39 Time:  0:07:39


Total images           : 5153
Correct predictions    : 4415
Incorrect predictions  : 738
Accuracy               : 85.67824568212691 %
